In [112]:
from pathlib import Path
import pandas as pd

In [113]:
data_file = Path.cwd() / 'comment_soup.txt'

In [ ]:
temp_data = list()
with data_file.open('r') as f:
    comments = f.read()
    for line in comments.split('\n\n'):
        line = line.strip()
        if line:
            line = line.replace('п»ї', ' ')
            label, author, text = line.split('|')
            temp_data.append(dict(label=label, author=author, text=text))

In [111]:
data = pd.DataFrame(temp_data)

In [92]:
data

,author,label,text
0,Berlino10,trolling,Sie sollten bei der Wahrheit bleiben.\nBeide P...
1,Mandalore,trolling,"Naja, seit dem Angebot von Hilfsgeldern seiten..."
2,Jo408039,good,Imperialismus? ein dummer und abgedroschener B...


In [93]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.base import TransformerMixin, BaseEstimator

In [94]:
data_train, data_test, label_train, label_test = train_test_split(data['text'].values, data['label'].values, test_size=0.3)

In [95]:
class TextCleaner(TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, texts, y=None):
        texts = [text.lower() for text in texts]
        return texts

In [96]:
features_pipe = Pipeline([
    ('cleaning', TextCleaner()),
    ('counts', CountVectorizer()),
])

pipe = Pipeline([
    ('features', features_pipe),
    ('nb', MultinomialNB()),
])

In [97]:
pipe.fit(data_train, label_train)

Pipeline(memory=None,
     steps=[('features', Pipeline(memory=None,
     steps=[('cleaning', <__main__.TextCleaner object at 0x000002B40FB422E8>), ('counts', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, ma...izer=None, vocabulary=None))])), ('nb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [106]:
pipe.predict(data_test)

array(['trolling'], 
      dtype='<U8')

In [107]:
data_test

array([ 'Sie sollten bei der Wahrheit bleiben.\nBeide PrГ¤sidentenwahlen wurden berechtigterweise gewonnen.In der Ukraine bestГ¤tigte das u.a. die OSZE.In RUS die Opposition, Golem, die u.a. einen aufwendigen SMS-Gegencheck mit Гјber 30 Mio Kontroll-Eingaben getГ¤tigt hat, sowie das unabhГ¤ngige Levada-Institut, Гјber 60% gehen ok, vielleicht nicht die 63%Bei den Parlamentswahlen in der Ukraine gibt es tatsГ¤chliche Betrugsnachweise.Bei denen in RUS auch, u.a. unverfГ¤lschbare rein statistische UnmГ¶glichkeiten. Einiges RUS hГ¤tte nach tatsГ¤chlichem WГ¤hlerwillen von 65% auf 35% abstГјrzen mГјssen statt der off. 49%.'], dtype=object)

In [109]:
new_comments = [
    'Propaganda.Ich glaube eher, dass die westlichen Medien (vor allem Spiegel-Online und amerikanische Medien) eine Anti-Putin Propaganda-Show abziehen. Sachlich ist das schon länger nicht mehr. Und diese ZEIT-Kommentarseite lebt sicherlich von These und Antithese, wie es schon die alten Griechen gemacht haben. Das hat mit "Sprachrohr" oder Propaganda nichts zu tun. Es ist einfach nur ein Standpunkt, den ich hier vertrete, weil ich meine, dass dieser Standpunkt bisher nicht genügend diskutiert wurde.'
]

In [115]:
pipe.predict(new_comments)

array(['good'], 
      dtype='<U8')